In [ ]:
from config_jupyter import set_wd
set_wd()
import modules.scrap_indec as scrap

In [ ]:
scrap.scrap_database(anio_desde="2019", anio_hasta="2019",frecuencia="anual")

In [ ]:
df = scrap.extraer_y_concatenar_dfs_anuales(opciones=["2019"],anio_hasta="19")

In [ ]:
import pandas as pd
enmienda = pd.read_csv("data/CE_ENMIENDA.csv", encoding="latin-1", sep=";",dtype={"ncm":str,"cnro_enmienda":str})
enmienda_dict = dict(zip(enmienda.ncm, enmienda.ncm_descri))
pais_origen = pd.read_csv("data/CE_PAIS.csv", sep=";",dtype=dict(CCOD_PAIS=str,CCOD_ASOC=str))
pais_origen.CCOD_ASOC = pais_origen.CCOD_ASOC.fillna(pais_origen.CCOD_PAIS)
dict_paises = dict(zip(pais_origen.CCOD_PAIS, pais_origen.CDESCRI))
pais_origen['pais_descri'] = pais_origen.CCOD_ASOC.apply(lambda x: dict_paises[x])
dict_paises = dict(zip(pais_origen.CCOD_PAIS, pais_origen.pais_descri))
dict_continentes = {'1':"África","2":"América","3":"Asia","4":"Europa","5":"Oceanía","9":"Indeterminado"}

In [ ]:
df.insert(2, "ncm_descri", df.ncm.apply(lambda x: enmienda_dict[x]))
df.insert(4,"pais_descri", df.porg.apply(lambda x: dict_paises[x]))
df.insert(5,"continente", df.porg.apply(lambda x: dict_continentes[x[0]]))

In [ ]:
df

In [ ]:
def calculos(df:pd.DataFrame):
    df["fle_fob"] = df["flete(u$s)"] / df["fob(u$s)"]
    df["fle_unitario"] = df["flete(u$s)"] / df["pnet(kg)"] * 1_000_000
    return df

In [ ]:
calculos(df).sort_values("fle_fob",ascending=False).head(20) # hay valores muy bajos que generan outliers

In [ ]:
sum(df["flete(u$s)"]) / sum(df["fob(u$s)"])

In [ ]:
calculos(df).columns

In [ ]:
import plotly.graph_objects as go

def plot_observaciones():
    df_plot = df.copy()
    df_plot = calculos(df_plot)
    fig = go.Figure(data=go.Scatter(
    x=df_plot["fob(u$s)"]/1_000_000,
    y=df_plot['flete(u$s)']/1_000_000,
    mode='markers',
    
    customdata=df[["ncm", "ncm_descri", "pais_descri", "fle_fob", "fle_unitario"]],
    hovertemplate=
    "<b>FOB:</b> %{x}<br>" +
    "<b>Flete:</b> %{y}<br>"+
    "<b>NCM:</b> %{customdata[0]}<br>" +
    "<b>Descripción NCM:</b> %{customdata[1]}<br>" +
    "<b>País:</b> %{customdata[2]}<br>" +
    "<b>Flete / FOB:</b> %{customdata[3]}<br>" +
    "<b>Flete Unitario:</b> %{customdata[4]}<br>" +
    "<extra></extra>"
    
    # customdata=[
    #     df_plot["ncm"], df_plot["ncm_descri"], df_plot["pais_descri"], df_plot["fle_fob"], df_plot["fle_unitario"]
    # ],
    # hovertemplate=
    # '<b>ncm:</b> %{customdata[0]}<br>' +
    # '<b>ncm_descri:</b> %{customdata[1]}<br>' +
    # '<b>pais_descri:</b> %{customdata[2]}<br>' +
    # '<b>fle_fob:</b> %{customdata[3]}<br>' +
    # '<b>fle_unitario:</b> %{customdata[4]}<br>',
))

    fig.update_yaxes(tickformat = ",")
    fig.update_xaxes(tickformat = ",")
    fig.update_layout(
        template = None,
        xaxis_title='fob(u$s)',
        yaxis_title='flete(u$s)',
        title='Gráfico Scatter',
        separators = ",."
    )
    return fig

# plot_observaciones()

In [ ]:
def plot_observaciones():
    df_plot = df.copy()
    df_plot = calculos(df_plot)
    df_plot = df_plot[(df_plot["fob(u$s)"] >= 1_000_000)&(df_plot.continente != "Indeterminado")&
                      (df.pais_descri != "Argentina")]
    fig = go.Figure(data=go.Scatter(
    x=df_plot["fob(u$s)"]/1_000_000,
    y=df_plot['fle_fob'],
    mode='markers',
    
    customdata=df_plot[["ncm", "ncm_descri", "pais_descri", "fob(u$s)", "fle_unitario"]],
    hovertemplate=
    "<b>Flete / FOB:</b> %{y}<br>" +
    "<b>Flete:</b> %{x}<br>"+
    "<b>NCM:</b> %{customdata[0]}<br>" +
    "<b>Descripción NCM:</b> %{customdata[1]}<br>" +
    "<b>País:</b> %{customdata[2]}<br>" +
    "<b>FOB:</b> %{customdata[3]}<br>" +
    "<b>Flete Unitario:</b> %{customdata[4]}<br>" +
    "<extra></extra>"
    
    # customdata=[
    #     df_plot["ncm"], df_plot["ncm_descri"], df_plot["pais_descri"], df_plot["fle_fob"], df_plot["fle_unitario"]
    # ],
    # hovertemplate=
    # '<b>ncm:</b> %{customdata[0]}<br>' +
    # '<b>ncm_descri:</b> %{customdata[1]}<br>' +
    # '<b>pais_descri:</b> %{customdata[2]}<br>' +
    # '<b>fle_fob:</b> %{customdata[3]}<br>' +
    # '<b>fle_unitario:</b> %{customdata[4]}<br>',
))

    fig.update_yaxes(tickformat = ",")
    fig.update_xaxes(tickformat = ",")
    fig.update_layout(
        template = None,
        xaxis_title='fob(u$s)',
        yaxis_title='flete / fob',
        title='Gráfico Scatter',
        separators = ",."
    )
    return fig

plot_observaciones()

In [ ]:
df[df["fob(u$s)"] >= 1_000_000].sort_values("fle_fob",ascending=False).describe()

In [ ]:
df[df["fob(u$s)"] >= 1_000_000].sort_values("fle_fob",ascending=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
def boxplot():
    df_plot = df.copy()
    df_plot = calculos(df_plot)
    df_plot = df_plot[(df_plot["fob(u$s)"] >= 1_000_000)&(df_plot.continente != "Indeterminado")&
                      (df.pais_descri != "Argentina")]
    
    # Crea el boxplot utilizando Seaborn
    plt.figure(figsize=(8, 6))
    sns.boxplot(data=df_plot, x="continente", y="fle_fob")

    # Personaliza el gráfico
    plt.title("Boxplot del Flete FOB por Continente")
    plt.xlabel("Continente")
    plt.ylabel("Flete / FOB")
    
    return plt.show()

boxplot()


In [ ]:
import pandas as pd
import plotly.express as px

def boxplot():
    df_plot = df.copy()
    df_plot = calculos(df_plot)
    df_plot = df_plot[(df_plot["fob(u$s)"] >= 1_000_000)&(df_plot.continente != "Indeterminado")&
                      (df.pais_descri != "Argentina")]
    
    # Crea el gráfico de boxplot utilizando Plotly Express
    fig = px.box(df_plot, x="continente", y="fle_fob",color="continente", hover_data=["ncm", "ncm_descri","pais_descri"])
    fig.update_yaxes(range=[0,0.3])
    # Personaliza el gráfico
    fig.update_layout(
        title="Boxplot del Flete FOB por Continente",
        xaxis_title="Continente",
        yaxis_title="Flete / FOB",
        template = None
    )
    
    # Muestra el gráfico
    fig.show()

boxplot()
